<a href="https://colab.research.google.com/github/Deepak-Work/Sovereign-Default-Prediction/blob/main/FRM_Classifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/content/master_data_1999_2021_comm_countries_withlabels_andtargets.csv")

<ipython-input-2-e45e09ae9e36>:1: DtypeWarning: Columns (259,287,289,292,294) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/master_data_1999_2021_comm_countries_withlabels_andtargets.csv")


In [4]:
data.head()

,"Exchange Rates, Nominal Effective Exchange Rate, Index","Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index","Exchange Rates, SDR per US Dollar, End of Period","Exchange Rates, SDR per US Dollar, Period Average","Exchange Rates, SDRs per ounce of Gold, End of period, Rate","Exchange Rates, US Dollar per SDR, End of period, Rate","Exchange Rates, US Dollar per SDR, Period average, Rate","Exchange Rates, US Dollars per ounce of gold, End of period, Rate","External Trade, Goods, Value of Exports, Free on Board (FOB), US Dollars","External Trade, Goods, Value of Imports, Cost, Insurance, Freight (CIF), US Dollars",...,d_sy_crit1,d_sy_crit2,d_sy_crit3,d_sy_crit4,d_dy,d_dy_crit1,d_dy_crit2,d_dy_crit3,d_dy_crit4,target
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.510000e+08,1.140000e+09,...,0.0,0.0,0.0,NaN,1.0,1.0,1.0,0.0,NaN,0
1,124.975154,125.228206,NaN,NaN,NaN,NaN,NaN,NaN,1.252500e+10,9.162000e+09,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.157000e+09,3.109000e+09,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1
3,106.649486,116.995165,NaN,NaN,NaN,NaN,NaN,NaN,3.800000e+07,4.140000e+08,...,0.0,0.0,0.0,NaN,1.0,1.0,0.0,0.0,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.333300e+10,2.550800e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [5]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [6]:
for col in data.columns:
  data[col] = pd.to_numeric(data[col], errors='coerce')

In [7]:
data = data.fillna(data.mean())

In [8]:
# 1. Drop columns with less than 25% data available
threshold = 0.25 * len(data)  # 25% of the number of rows
data = data.dropna(thresh=threshold, axis=1)

# Separating features and target
X = data.drop('target', axis=1)
y = data['target']

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# 2. Upsampling the minority class in the training set
# Note: Upsampling should be done only on training data to prevent information leakage
minority_class = y_train == y_train.value_counts().idxmin()
X_upsampled, y_upsampled = resample(X_train[minority_class],
                                    y_train[minority_class],
                                    replace=True,
                                    n_samples=X_train[~minority_class].shape[0],
                                    random_state=42)

X_train_upsampled = pd.concat([X_train[~minority_class], X_upsampled])
y_train_upsampled = pd.concat([y_train[~minority_class], y_upsampled])


In [11]:
# 3. Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_upsampled)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Train SVM model
svm_model = SVC(kernel='linear',probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train_upsampled)

# Make predictions and evaluate the SVM model
svm_predictions = svm_model.predict(X_test_scaled)
print('SVM Model Performance:')
print('Accuracy:', accuracy_score(y_test, svm_predictions))
print(classification_report(y_test, svm_predictions))

SVM Model Performance:
Accuracy: 0.9985422740524781
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       640
           1       0.98      1.00      0.99        46

    accuracy                           1.00       686
   macro avg       0.99      1.00      0.99       686
weighted avg       1.00      1.00      1.00       686



In [16]:
# Train KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train_upsampled)

# Make predictions and evaluate the KNN model
knn_predictions = knn_model.predict(X_test_scaled)
print('KNN Model Performance:')
print('Accuracy:', accuracy_score(y_test, knn_predictions))
print(classification_report(y_test, knn_predictions))

KNN Model Performance:
Accuracy: 0.8775510204081632
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       640
           1       0.26      0.46      0.33        46

    accuracy                           0.88       686
   macro avg       0.61      0.68      0.63       686
weighted avg       0.91      0.88      0.89       686



Let's examine the performance of both models based on the metrics provided:

### SVM Model Performance:
- **Accuracy:** The SVM model has a very high accuracy of approximately 99.85%. However, accuracy is not always the best measure, especially if the classes are imbalanced.
- **Precision:** Precision for the majority class (class 0) is perfect (1.00), and very high for the minority class (class 1) at 0.98.
- **Recall:** Recall is also perfect for the majority class and very high for the minority class (1.00), indicating the model is correctly identifying all positive cases.
- **F1-Score:** The F1-score, which balances precision and recall, is perfect for the majority class and very high for the minority class (0.99).

Overall, the SVM model is performing exceptionally well across all metrics. The high F1-score for the minority class is particularly impressive, suggesting the model is performing well in identifying the minority class despite potential class imbalance.

### KNN Model Performance:
- **Accuracy:** The KNN model has a lower accuracy of approximately 87.75%.
- **Precision:** There is a large discrepancy in precision between the majority class (0.96) and the minority class (0.26). This suggests that when the model predicts an instance as class 1, it is correct only 26% of the time.
- **Recall:** Recall for the majority class is lower than perfect at 0.91, indicating some instances of class 0 are being missed. The recall for the minority class is higher (0.46), indicating it's better at catching true positives of class 1 than the precision suggests.
- **F1-Score:** The F1-score for the majority class is high (0.93), but much lower for the minority class (0.33), showing a moderate performance for the minority class.

The KNN model is not performing as well as the SVM, particularly in terms of precision and F1-score for the minority class. The higher recall for the minority class at the expense of precision suggests the model may be over-predicting the minority class.

**Conclusion:**
The SVM model is performing excellently and is well-suited to this dataset, managing to identify the minority class without sacrificing precision or recall. On the other hand, the KNN model is struggling, particularly with precision for the minority class, which leads to a lower F1-score.

It's worth noting that these results are based on the test set. It's important to ensure that these metrics are not the result of overfitting and that the model generalizes well to unseen data. Cross-validation could be used to help validate these results. If the dataset is highly imbalanced, it would also be worthwhile to look at the confusion matrix or other metrics such as the ROC-AUC score.

In [17]:
from sklearn.metrics import roc_auc_score

# SVM model - make sure you have trained your SVM with probability=True
# Predict probabilities for the test data
svm_probs = svm_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for class 1

# Compute the ROC-AUC score
svm_roc_auc = roc_auc_score(y_test, svm_probs)
print(f'SVM ROC-AUC Score: {svm_roc_auc:.2f}')

# KNN model - KNN natively provides probabilities so there is no need to adjust the model initialization
# Predict probabilities for the test data
knn_probs = knn_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for class 1

# Compute the ROC-AUC score
knn_roc_auc = roc_auc_score(y_test, knn_probs)
print(f'KNN ROC-AUC Score: {knn_roc_auc:.2f}')


SVM ROC-AUC Score: 1.00
KNN ROC-AUC Score: 0.74


In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Assuming you have a DataFrame `data` and have already prepared `X` (features) and `y` (target)
# For this example, let's assume `X` is not yet upscaled or scaled, and `y` is your target variable.

# Upsample your minority class here if you haven't done so already
# Note: Ideally, upsampling should be done within each cross-validation fold as well

# Initialize models
svm_model = SVC(kernel='linear', probability=True, random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=5)

# Define a stratified K-fold cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create pipelines for both models to include scaling
svm_pipeline = make_pipeline(StandardScaler(), svm_model)
knn_pipeline = make_pipeline(StandardScaler(), knn_model)

# Perform cross-validation
svm_cross_val_scores = cross_val_score(svm_pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
knn_cross_val_scores = cross_val_score(knn_pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)

# Print ROC-AUC scores for each fold and the mean score for SVM
print(f'SVM Cross-Validation ROC-AUC Scores: {svm_cross_val_scores}')
print(f'SVM Mean Cross-Validation ROC-AUC Score: {np.mean(svm_cross_val_scores):.2f}')

# Print ROC-AUC scores for each fold and the mean score for KNN
print(f'KNN Cross-Validation ROC-AUC Scores: {knn_cross_val_scores}')
print(f'KNN Mean Cross-Validation ROC-AUC Score: {np.mean(knn_cross_val_scores):.2f}')


SVM Cross-Validation ROC-AUC Scores: [1. 1. 1. 1. 1.]
SVM Mean Cross-Validation ROC-AUC Score: 1.00
KNN Cross-Validation ROC-AUC Scores: [0.91992014 0.94065293 0.92860846 0.92072132 0.92386877]
KNN Mean Cross-Validation ROC-AUC Score: 0.93
